<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_W01_Turing_Test_Practical_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 1 Lab:** Turing Test Practical — Build a Rule‑based Chatbot

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> In this lab you will create and extend a tiny ELIZA‑style chatbot, then run a mini **Turing Test**.
> Every code cell is commented line‑by‑line so you understand exactly what it does.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.
2. Run the **Setup** cell once.
3. Work through the tasks. Edit only cells labeled **`# TODO(Student)`**.
4. Use the **Quick Checks** to test your code.
5. Finish with the **Reflection**. If you are done early, try the **Extensions**.

**Assessment focus:** clarity of rules, correctness of memory feature, and quality of your transcript.

In [ ]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# This cell collects your group information using Colab form fields.
# The '#@title' line makes a nice header above the cell UI.
# The '#@param' annotations create text boxes in Colab for easy input.

group_members = "David, "  #@param {type:"string"}  # Names of all group members (comma‑separated)
scenario = "One sentence describing your chatbot use‑case (e.g., student help desk)"  #@param {type:"string"}  # Your real‑world scenario

# The print statements give immediate confirmation of what you entered.
print("👥 Group members:", group_members)  # Show the group roster
print("🧩 Scenario:", scenario)            # Show the chosen scenario

## Do Now (2–3 mins)
Discuss in pairs: *When might a rule‑based chatbot be convincing? When might it fail?*  
Add one sentence to your **Group Log** that names your use‑case (e.g., timetable Q&A).

In [ ]:
#@title 🔧 Setup (run once per session)
# This cell ensures any needed packages are available in Colab.
# For Week 1 we only use Python's standard library modules (no extra installs).

import sys  # Gives access to Python interpreter details (not strictly needed here)
import re   # Regular expressions for pattern matching (core of our rule system)
import random  # To randomly pick between multiple plausible responses

# We show a simple confirmation message so you know setup ran successfully.
print("✅ Setup complete. You're ready to build a chatbot!")

## Mini‑notes: How this chatbot works
A rule‑based chatbot uses **pattern matching** to map input text to template responses.  
We’ll write a small set of regular‑expression rules and a **fallback** for anything unmatched.

In [ ]:
rules = [ 
    (re.compile(r'\b(hello|hi|hey)\b', re.I),
     lambda m: random.choice(["Hello! How can I help today?", "Hi there—what's on your mind?"])),
    
    (re.compile(r'i\s+feel\s+(.*)', re.I),
     lambda m: f"Why do you feel {m.group(1).strip()}?"),
    
    (re.compile(r'.*\?$', re.I),
     lambda m: "That's a good question. What do you think?"),
]

def eliza_reply(text: str) -> str:
    text = text.strip()
    for pattern, handler in rules:
        match = pattern.search(text)
        if match:
            return handler(match)
    return random.choice([
        "Tell me more…",
        "Hmm, interesting. Go on.",
        "How does that make you feel?",
        "I'm listening — keep going!"
    ])

# ==============================
# MEMORY + SMART CONTENT-BASED RESPONSES
# ==============================

user_name = None  # Remembers your name


## Task 1 — Extend the chatbot with **memory** and two new rules
1. If the user says **“my name is …”**, store the name and greet them personally next time.  
2. Add **two** more patterns that would be useful for your scenario (e.g., thanks/bye, course code questions).  
3. Keep your explanations in the comments so future you (and your teammates) understand the code.

In [ ]:
# TODO(Student): Add memory + at least two new rules
# We implement a new function 'eliza_reply_plus' that wraps the original 'eliza_reply'
# and adds: (a) name memory and (b) two extra pattern handlers.

# A global variable (for simplicity) holds the user's name if provided.
user_name = None  # Initially, we don't know the user's name

def eliza_reply_plus(text: str) -> str:
    """Extended version with name memory and extra rules."""
    global user_name                      # We will update the global 'user_name' variable
    text = text.strip()                   # Normalize input by trimming whitespace

    # --- New Rule A: Capture and remember the user's name --------------------
    # Pattern: 'my name is <word>'. We capture the name in group(1).
    m = re.search(r'\bmy name is ([A-Za-z]+)\b', text, re.I)  # Look for the name phrase (letters only for simplicity)
    if m:                                                       # If the pattern matched
        user_name = m.group(1).title()                          # Store a title-cased version of the name
        return f"Nice to meet you, {user_name}. How can I help?"  # Personal greeting

    # If we already know the user's name and they greet us, greet them back personally.
    if user_name and re.search(r'\b(hello|hi|hey)\b', text, re.I):  # Detect a greeting with known name
        return f"Hi {user_name}! What's next?"                       # Personalized response

    # --- New Rule B: Gratitude detection -------------------------------------
    # Many conversations include 'thank you' or 'thanks' — acknowledge politely.
    if re.search(r'\b(thank you|thanks)\b', text, re.I):             # Detect gratitude words
        return "You're welcome! Is there anything else I can help with?"  # Friendly acknowledgement

    # --- New Rule C: Goodbye detection ---------------------------------------
    # End the conversation gracefully when the user says 'bye' or 'goodbye'.
    if re.search(r'\b(bye|goodbye)\b', text, re.I):                  # Detect common goodbye variations
        return "Goodbye! Have a great day."                           # Courteous closing

    # If none of the new rules apply, fall back to the original rule set.
    return eliza_reply(text)                                           # Delegate to base rules

### Quick Check — Guided tests
Run the cell below to make sure your memory rule works.

In [ ]:
SMART_RESPONSES = {
    "cs101": "CS101 is Intro to Python! We cover: variables, loops, conditionals, functions, lists, dictionaries, file I/O, and a final project (usually a game or data analyzer).",
    "exam": "The exam is 60% of your grade: 40 MCQs + 3 coding questions. Topics: functions, loops, lists, strings, and file handling. Practice past papers!",
    "past paper": "Check the course website → Resources → Exam Prep. All past exams from 2020–2024 are there!",
    "deadline": "Assignment 3 is due Friday 11:59 PM on Canvas. No extensions without medical proof.",
    "assignment": "Yes! Assignment 3 is about reading/writing files and using functions. Use the starter code — it has helpful comments!",
    "function": "A function is a reusable block of code. Example:\n\ndef greet(name):\n    return f'Hello {name}!'\n\nCall it with: greet('Sofia')",
    "loop": "Loops repeat code. 'for' loop example:\nfor i in range(5):\n    print(i)  → prints 0 to 4",
    "list": "Lists store multiple items: fruits = ['apple', 'banana', 'cherry']\nAccess with index: fruits[0] → 'apple'",
    "stressed": "Totally normal! Try this: study 25 min → break 5 min (Pomodoro). After 4 rounds, take 15–30 min off. It really helps!",
    "overwhelmed": "Start small. Pick ONE thing (e.g., read 1 page). Once you start, it gets easier. You’ve got this!",
    "confused": "No shame in that! Can you paste the exact line/code/question that’s confusing? I’ll explain it simply.",
    "don't understand": "Which part? Functions? Loops? Just tell me the topic and I’ll break it down from zero.",
    "tired": "Sleep > cramming. One all-nighter can drop your grade 10–20%. Rest, then study — you’ll learn faster!",
    "motivate": "You’re already ahead because you’re asking for help. Keep going — future you will be so grateful!",
    "thank": random.choice(["You're very welcome!", "Happy to help!", "Anytime!", "My pleasure!"]),
    "thanks": "You're so welcome! Keep up the amazing work!",
}

def eliza_reply_plus(text: str) -> str:
    global user_name
    original = text
    text = text.strip()
    lower = text.lower()

    # 1. Remember name
    name_match = re.search(r'\bmy name is\s+([a-zA-Z]+)', lower)
    if name_match:
        user_name = name_match.group(1).capitalize()
        return f"Hi {user_name}! Nice to meet you! I'm your personal CS101 study buddy. What can I help with?"

    # 2. Personalized greeting
    if user_name and re.search(r'\b(hello|hi|hey|morning|afternoon|evening)\b', lower):
        return random.choice([
            f"Hey {user_name}! Welcome back!",
            f"Hi again, {user_name}! Ready to study?",
            f"Good to see you, {user_name}!",
            f"What's up, {user_name}? How can I help today?"
        ])

    # 3. Smart keyword responses
    for keyword, response in SMART_RESPONSES.items():
        if keyword in lower:
            if callable(response):
                return response()
            return response

    # 4. Goodbye
    if re.search(r'\b(bye|goodbye|see you|later|good night|ttyl)\b', lower):
        if user_name:
            return random.choice([
                f"Bye {user_name}! Study smart, not hard!",
                f"See you soon, {user_name}! You've got this!",
                f"Take care, {user_name}! Come back anytime!"
            ])
        return "Goodbye! Good luck with your studies!"

    # 5. Fallback to ELIZA
    return eliza_reply(original)

# ==============================
# INTERACTIVE CHAT MODE (This is what you wanted!)
# ==============================


## Task 2 — **Turing Test** Activity (10–15 min)
1. One student chats with the bot for ~1 minute while others watch the transcript.  
2. Observers decide if it *could* be human and list **what made it convincing** and **what gave it away**.  
3. Rotate roles and try to improve your rules based on feedback.

In [ ]:
print("="*60)
print("       CS101 STUDY BUDDY IS ONLINE!")
print("   (Type 'quit', 'exit', or 'bye' to stop)")
print("="*60)
print("You can say things like:")
print("   • My name is Alex")
print("   • What is CS101 about?")
print("   • I'm stressed about the exam")
print("   • When is the assignment due?")
print("   • Explain functions")
print("="*60)

while True:
    try:
        user_input = input("\nYou → ").strip()
        
        if user_input.lower() in ["quit", "exit", "bye", "goodbye"]:
            if user_name:
                print(f"Bot → Goodbye {user_name}! See you next time!")
            else:
                print("Bot → Goodbye! Study well!")
            break
            
        if user_input == "":
            print("Bot → You didn't say anything! Type something :)")
            continue
            
        response = eliza_reply_plus(user_input)
        print(f"Bot → {response}")
        
    except KeyboardInterrupt:
        print("\n\nBot → Okay, see you later!")
        break
    except EOFError:
        break

## Reflection (5–7 sentences)
- When did the bot seem **convincing**? When did it obviously fail?  
- Which two rules mattered most for your scenario?  
- What would you change next time (e.g., add context, handle negation, track topics)?

In [ ]:
#@title 📝 Reflection (edit the text inside the triple quotes)
reflection = """
Write your 5–7 sentence reflection here.
- What felt realistic? What broke the illusion?
- How did your 'memory of name' change the tone of the conversation?
- Which additional rule would deliver the biggest improvement for your chosen scenario?
"""

# Printing helps ensure your reflection is saved in the notebook output.
print(reflection.strip())  # Show the reflection text as entered by the student

## Extensions (if you finish early)
- Add a **topic tracker** (e.g., if the user mentions 'exam', ask a follow‑up).  
- Implement a tiny **FAQ table** (dictionary) for your scenario (e.g., office hours).  
- Log the last **3 messages** to create context for follow‑up questions.